# Introduccion a Langchain

## Antes de comenzar...

Durante a lo largo de este repositorio estare hablando acerca de lo aprendido en langchain y como estuve trabajando con el, es por ello que sera necesario correr los modelos de manera local.
En este caso estaremos haciendo el uso de Ollama y estaremos descargando algunos modelos segun se vayan necesitando.
Para consultar mas informacion acerca de Ollama presiona [aqui.](https://github.com/Sanchez1909/Introduccion-a-Langchain/blob/main/Ollama/Ollama.md)

## Componentes esenciales

Para llevar poder llevar acabo este curso, sera necesario tener instalado:
- Visual Studio Code
- Ollama
    - nomic-embed-text (Nos servira para trabajar con los embeddings)
    - llama3.2 / deepseek-r1:1.5b
- Python
- Jupyter Notebook/Google Colab

---

## Navegando por langchain

En estos momentos, estaremos navegando por diferentes componentes de langchain para poder conocer sus diversas funcionalidades y poder trabajar con la biblioteca.

---

### Chat Models

Estos componentes son abstracciones que encapsulan la interaccion con modelos de lenguaje de gran escala (LLM), facilitando la integracion de estos modelos en nuestras aplicaciones y añadiendoles diversas funcionalidades.

In [11]:
!pip install langchain_ollama -q


In [18]:
import ollama 
from langchain_ollama import ChatOllama 

In [19]:
llm = ChatOllama(model="llama3.2")
respuesta = llm.invoke("Cuentame un chiste")
print(respuesta)

content='¿Por qué la computadora fue al doctor?\n\nPorque tenía un virus! (risas)\n\nEspero que te haya hecho sonreír. ¿Quieres escuchar otro?' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-02-07T19:06:19.2221659Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 27304272900, 'load_duration': 16659915200, 'prompt_eval_count': 31, 'prompt_eval_duration': 2249189000, 'eval_count': 39, 'eval_duration': 8345684000} id='run-e7d54938-b48b-44f9-bbf5-3bbeb5c8b89a-0' usage_metadata={'input_tokens': 31, 'output_tokens': 39, 'total_tokens': 70}


> Importante tener en cuenta que el tiempo de procesamiento de la respuesta depende de la capacidad de nuestra computadora y del tamaño del modelo.

### Prompt Templates

Los Prompt Templates son plantillas que nos facilitan la generacion de prompts para poder unirlos con los modelos de lenguaje y conseguir resultados mas especificos.

Son muy utilices, ya que son bastante flexibles y nos permiten que la necesidad de su rediseño sea minima.

In [ ]:
!pip install langchain.prompts

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
promptTemplate = PromptTemplate.from_template()